In [ ]:
# ROBERT HEETER
# ELEC 378 Machine Learning
# 24 February 2023

# PROBLEM SET 6


In [ ]:
# PROBLEM 1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PART A

df_train = pd.read_csv('train.csv')
train_nums = df_train.select_dtypes(include='number') # select only numerical data types
del train_nums['MSSubClass'] # the MSSubClass column is not actually a numerical feature, so remove it

data = train_nums.values
X = data[:,~np.isnan(data).any(axis=0)]
y = X[:,-1] # final column of the data set is the actual sale price
X = np.delete(X,-1,1)

m = X.min(axis=0) # minimum value in each feature
r = np.ptp(X,axis=0) # range of values for each feature

X = (X-m)/r # normalize data using minimum value and range of values

X[:,0] = np.ones([len(X)]) # set first column to 1's to account for the center of the data; this replaces the first column of house indexes in the dataset, which is not informative

print('dataset for inspection:')
print(X)
print(np.shape(X))

In [ ]:
# PART B

# scratch linear regression implementation
psuedo_X = np.linalg.pinv(X) # compute Moore-Penrose pseudoinverse, pseudo_X = (X^T*X)^-1 * X^T

w = np.matmul(psuedo_X, y) # find optimal weightings, w = pseudo_X*y
y_approx = np.matmul(X, w) # find calculated y (sale price) from regression, y_approx = X*w (+ error)

n = len(y)
error = np.mean((np.abs(y-y_approx))/y)*100
# error = np.linalg.norm(y-y_approx,ord=1)/n # use 1-norm to calculate average deviation from

print('scratch linreg predicted price (y_approx) average percent difference from actual price (y):')
print(f'{round(error, 3)}%')

plt.figure(0)
plt.scatter(y/1000, y_approx/1000, s=5)
plt.plot([0,700], [0,700], 'r-', linewidth=1)
plt.legend(['home','perfect accuracy (prediction = actual)'])
plt.xlabel('Actual sale price [$1000s]')
plt.ylabel('Linreg predicted sale price [$1000s]')
plt.title('Accuracy of linreg predicted sale price')

# check prediction using sklearn
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y) 
sk_y_approx = reg.predict(X)
sk_diff = np.mean((np.abs(sk_y_approx-y_approx))/y_approx)*100

print('\nsklearn predicted price (sk_y_approx) average percent difference from scratch linreg implementation (y_approx):')
print(f'{round(sk_diff, 16)}%')


In [ ]:
# PART C

w_i = np.flip(np.argsort(abs(w)))

train_nums = train_nums.rename(columns = {'Id':'~'}) # rename first column to '~' since it was used for centering
labels = train_nums.columns[~np.isnan(data).any(axis=0)] # remove all columns that were ignored for linear regression

print('most to least impactful features for determining sale price:')
print(labels[w_i])

print('\nmost to least impactful feature weights for determining sale price:')
print(w[w_i])
